In [1]:
%load_ext autoreload
%autoreload 2
import os
import re
import json
import copy
import sys
from tqdm import tqdm
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns

import pytrec_eval
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from haystack.pipelines import Pipeline
from haystack.nodes import BM25Retriever, ElasticsearchRetriever
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever, SentenceTransformersRanker
from haystack.nodes.reader import FARMReader
from haystack.utils import print_answers
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../../')
import globals
from elastic_search_utils import elastic_utils
from haystack_utils.retrievers import BioASQ_Retriever
import bioasq_eval
import haystack_util

working_folder = globals.PATH.home + '/data/working_folder'
eval_home = globals.PATH.eval_home + '/'
gs_google_docs = eval_home + '/examples/aueb_google_docs/aueb_nlp-bioasq6b-submissions/'
index_name = globals.BIOASQ.index + 'working_folder'
model_id = 'doc_retrieval_test'

es = Elasticsearch(globals.ES.server)

# set document store
document_store = ElasticsearchDocumentStore()

Home path : /opt/bioasq/col-un-bioasq11
Eval path : /opt/bioasq/Evaluation-Measures


In [3]:
test_batch_doc = f'{globals.PATH.home}/data/11b_testset/BioASQ-task11bPhaseA-testset2.json'
test_batch_json = json.load(open(test_batch_doc))

In [4]:
# set document store
document_store = ElasticsearchDocumentStore()
# create the retriever
retriever = BioASQ_Retriever(document_store = document_store)

# create the Sentence Transformer Ranker
ranker = SentenceTransformersRanker(model_name_or_path="cross-encoder/ms-marco-MiniLM-L-12-v2")

# create the Query Pipeline
pipeline = Pipeline()

# add bm25 retriever
pipeline.add_node(component=retriever, name="BM25Retriever", inputs=["Query"])
pipeline.add_node(component=ranker, name="Ranker", inputs=["BM25Retriever"])
my_model = "deepset/roberta-base-squad2"
reader = FARMReader(model_name_or_path=my_model, use_gpu=True)
pipeline.add_node(component=reader, name="Reader", inputs=["Ranker"])

# run the pipeline
prediction = pipeline.run(query="Which factors drive replisome disassembly during DNA replication termination and mitosis", params={"BM25Retriever": {"top_k": 100}})

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [5]:
prediction

{'query': 'Which factors drive replisome disassembly during DNA replication termination and mitosis',
 'no_ans_gap': 8.580459594726562,
 'answers': [<Answer {'answer': 'Cullin ubiquitin ligases', 'type': 'extractive', 'score': 0.9014731049537659, 'context': 'Cullin ubiquitin ligases drive replisome disassembly during DNA replication termination. In worm, frog and mouse cells, CUL2LRR1 is required to ubiqui', 'offsets_in_document': [{'start': 0, 'end': 24}], 'offsets_in_context': [{'start': 0, 'end': 24}], 'document_ids': ['34195792'], 'meta': {'title': 'Reconstitution of human CMG helicase ubiquitylation by CUL2LRR1 and multiple E2 enzymes.', 'issue': '478(14)', 'pages': '2825-2842', 'abstract': 'Cullin ubiquitin ligases drive replisome disassembly during DNA replication termination. In worm, frog and mouse cells, CUL2LRR1 is required to ubiquitylate the MCM7 subunit of the CMG helicase. Here, we show that cullin ligases also drive CMG-MCM7 ubiquitylation in human cells, thereby making

In [6]:
def get_snipet_from_answer(pred):
    is_extractive = pred.type == 'extractive'
    answer = {}
    answer['document'] =  globals.BIOASQ.doc_relative_url + pred.document_ids[0]
    answer['text'] = pred.context
    answer['offsetInBeginSection'] = pred.meta['abstract'].rfind(pred.context)
    answer['offsetInEndSection'] = answer['offsetInBeginSection'] + len(pred.context)
    answer["beginSection"] = "abstract",
    answer["endSection"] = "abstract"
    return answer
    
for sample in tqdm(test_batch_json['questions'], position=0):
    prediction = pipeline.run(query=sample['body'], params={"BM25Retriever": {"top_k": 100}, "Ranker": {"top_k": 30}, "Reader": {"top_k": 30} })
    snippets = []
    doc_list_by_snnipets = []
    for ans in prediction['answers']:
        if ans.score > 0.3:
            snippet = get_snipet_from_answer(ans)
            doc_list_by_snnipets.append(snippet['document'])
            snippets.append(snippet)
            
    sample['snippets'] = snippets
    doc_list = [ globals.BIOASQ.doc_relative_url + d.id for d in prediction['documents'] ]
    #sample['documents'] = doc_list[0:10]
    sample['documents'] = list(set(doc_list_by_snnipets))
    
# mindlab base ans.score > 0.3:
# mindlab tns ans.score > 0.4 and only snnipets docs

  0%|                                                                                                                    | 0/75 [00:00<?, ?it/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

  1%|█▍                                                                                                          | 1/75 [00:05<06:26,  5.23s/it]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

  3%|██▉                                                                                                         | 2/75 [00:10<06:10,  5.07s/it]

...

 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████▏  | 73/75 [06:47<00:11,  5.55s/it]

Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 74/75 [06:52<00:05,  5.53s/it]

Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [06:58<00:00,  5.58s/it]


In [7]:
test_batch_json

{'questions': [{'id': '63f03006f36125a426000018',
   'type': 'summary',
   'body': 'What were the results of the DESTINY-Breast04 Trial?',
   'snippets': [{'document': 'http://www.ncbi.nlm.nih.gov/pubmed/35665782',
     'text': "st cancer, trastuzumab deruxtecan resulted in significantly longer progression-free and overall survival than the physician's choice of chemotherapy. ",
     'offsetInBeginSection': 2265,
     'offsetInEndSection': 2415,
     'beginSection': ('abstract',),
     'endSection': 'abstract'},
    {'document': 'http://www.ncbi.nlm.nih.gov/pubmed/36386213',
     'text': 'er (ABC). DESTINY-Breast04 (NCT03734029) has demonstrated the antitumor activity of trastuzumab deruxtecan (T-DXd). However, the balance between the e',
     'offsetInBeginSection': 162,
     'offsetInEndSection': 312,
     'beginSection': ('abstract',),
     'endSection': 'abstract'},
    {'document': 'http://www.ncbi.nlm.nih.gov/pubmed/31022475',
     'text': 'fter evaluating the results of these 6 